In [11]:
import pandas as pd
data_folder = r'../../survey_processing/processed_data/'
target='deprived_sev'
train_df = pd.read_csv(f'{data_folder}dhs_processed.csv').dropna(subset = [target])
available_imagery = []
imagery_path = r'/home/cs-yang3/rds/rds-t2-cs177-KNoCjrg2atA/wp3/data/satellite_imagery/esa_10'
imagery_source = 'S'
import os
for d in os.listdir(imagery_path):
    if d[-2] == imagery_source:
        for f in os.listdir(os.path.join(imagery_path, d)):
            available_imagery.append(os.path.join(imagery_path, d, f))

def is_available(centroid_id):
    for centroid in available_imagery:
        if centroid_id in centroid:
            return True
    return False
train_df = train_df[train_df['CENTROID_ID'].apply(is_available)]

def filter_contains(query):
    """
    Returns a list of items that contain the given query substring.
    
    Parameters:
        items (list of str): The list of strings to search within.
        query (str): The substring to search for in each item of the list.
        
    Returns:
        list of str: A list containing all items that have the query substring.
    """
    # Use a list comprehension to filter items
    for item in available_imagery:
        if query in item:
            return item
train_df['imagery_path'] = train_df['CENTROID_ID'].apply(filter_contains)
train_df

,CENTROID_ID,SURVEY_NAME,COUNTRY,YEAR,LATNUM,LONGNUM,cluster,health_vac_moddep,health_con_moddep,health_ari_moddep,...,hv009,hv105,hv216,v005,hv205_14,v312_17,v312_2,hv201_12,hv205_31,imagery_path
0,UG201600000001,Uganda_DHS_2016,Uganda,2016,0.320188,32.568206,1.0,0.000000,0.0,0.0,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.000000,/home/cs-yang3/rds/rds-t2-cs177-KNoCjrg2atA/wp...
1,UG201600000002,Uganda_DHS_2016,Uganda,2016,0.340653,32.593627,2.0,0.032258,0.0,0.0,...,0.148859,0.299020,0.065217,0.119688,0.0,0.0,0.0,0.0,0.166667,/home/cs-yang3/rds/rds-t2-cs177-KNoCjrg2atA/wp...
2,UG201600000003,Uganda_DHS_2016,Uganda,2016,0.313103,32.566556,3.0,0.052632,0.0,0.0,...,0.116183,0.058824,0.065217,0.085795,0.0,0.0,0.0,0.0,0.000000,/home/cs-yang3/rds/rds-t2-cs177-KNoCjrg2atA/wp...
3,UG201600000004,Uganda_DHS_2016,Uganda,2016,0.353368,32.558144,4.0,0.108108,0.0,0.0,...,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.000000,/home/cs-yang3/rds/rds-t2-cs177-KNoCjrg2atA/wp...
4,UG201600000005,Uganda_DHS_2016,Uganda,2016,0.367388,32.594357,5.0,0.034483,0.0,0.0,...,0.148133,0.435294,0.065217,0.146261,0.0,0.0,0.0,0.0,0.000000,/home/cs-yang3/rds/rds-t2-cs177-KNoCjrg2atA/wp...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24209,ET201900000301,Ethiopia_DHS_2019,Ethiopia,2019,9.614227,41.997375,301.0,0.054054,0.0,0.0,...,0.247087,0.535858,0.068791,0.005866,0.0,0.0,0.0,0.0,0.986301,/home/cs-yang3/rds/rds-t2-cs177-KNoCjrg2atA/wp...
24210,ET201900000302,Ethiopia_DHS_2019,Ethiopia,2019,9.554591,41.863785,302.0,0.098361,0.0,0.0,...,0.203657,0.295486,0.078868,0.008044,0.0,0.0,0.0,0.0,0.093023,/home/cs-yang3/rds/rds-t2-cs177-KNoCjrg2atA/wp...
24211,ET201900000303,Ethiopia_DHS_2019,Ethiopia,2019,9.524405,41.834015,303.0,0.042553,0.0,0.0,...,0.249230,0.496521,0.078541,0.009587,0.0,0.0,0.0,0.0,0.946237,/home/cs-yang3/rds/rds-t2-cs177-KNoCjrg2atA/wp...
24212,ET201900000304,Ethiopia_DHS_2019,Ethiopia,2019,9.514266,41.770584,304.0,0.029126,0.0,0.0,...,0.272525,0.478391,0.071872,0.005691,0.0,0.0,0.0,0.0,0.540816,/home/cs-yang3/rds/rds-t2-cs177-KNoCjrg2atA/wp...


In [18]:
import rasterio
import numpy as np
def get_features(path, normalization=3000.0):
    # Open the raster and read up to 13 bands
    with rasterio.open(path) as src:
        bands = src.read()[:13].astype(np.float32)
    
    # Normalize band values (optional, depends on sensor range)
    bands /= normalization
    
    # Replace NaNs and infinities with safe defaults, e.g., 0 or 1
    bands = np.nan_to_num(bands, nan=0.0, posinf=1.0, neginf=0.0)
    
    # -----------------------------------------------------------------
    # 1. Band-wise summary statistics
    # -----------------------------------------------------------------
    # Flatten each band to compute mean, std across all pixels
    band_means = bands.reshape(13, -1).mean(axis=1)
    band_stds  = bands.reshape(13, -1).std(axis=1)
    
    # -----------------------------------------------------------------
    # 2. Compute some common spectral indices
    #    (Adjust band indices to match your sensor)
    # -----------------------------------------------------------------
    # Example band indices for Sentinel-2-like data:
    #   Blue  = bands[1],  Green = bands[2],  Red = bands[3],  NIR  = bands[7]
    epsilon = 1e-6  # small constant to avoid divide-by-zero
    
    RED_IDX = 3
    NIR_IDX = 7
    GREEN_IDX = 2
    BLUE_IDX = 1

    red  = bands[RED_IDX]
    nir  = bands[NIR_IDX]
    green = bands[GREEN_IDX]
    blue  = bands[BLUE_IDX]
    
    # NDVI = (NIR - Red) / (NIR + Red)
    ndvi = (nir - red) / (nir + red + epsilon)
    # EVI  = 2.5 * (NIR - Red) / (NIR + 6*Red - 7.5*Blue + 1)
    evi  = 2.5 * (nir - red) / (nir + 6.0*red - 7.5*blue + 1.0 + epsilon)
    # NDWI = (Green - NIR) / (Green + NIR)
    ndwi = (green - nir) / (green + nir + epsilon)

    # Compute basic stats for these indices
    ndvi_mean, ndvi_std = ndvi.mean(), ndvi.std()
    evi_mean,  evi_std  = evi.mean(),  evi.std()
    ndwi_mean, ndwi_std = ndwi.mean(), ndwi.std()
    
    # -----------------------------------------------------------------
    # 3. Concatenate all features into one vector
    # -----------------------------------------------------------------
    # Collect band-level stats
    features = []
    for i in range(13):
        features.append(band_means[i])
        features.append(band_stds[i])
    
    # Add index-level stats
    features.extend([
        ndvi_mean, ndvi_std,
        evi_mean,  evi_std,
        ndwi_mean, ndwi_std
    ])
    
    # Return as a NumPy array (or dict if you prefer)
    return np.array(features, dtype=np.float32)

In [19]:
X = []
y = []
from tqdm import tqdm

for i, row in tqdm(train_df.iterrows(), total=len(train_df)):
    path = row['imagery_path']
    features = get_features(path)
    X.append(features)
    y.append(row[target])

  0%|          | 0/8650 [00:00<?, ?it/s]

100%|██████████| 8650/8650 [1:38:13<00:00,  1.47it/s]  


In [ ]:
# save the features and labels
X = np.array(X)
y = np.array(y)
np.save(f'X.npy', X)
np.save(f'y.npy', y)

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,y
0,0.457391,0.033506,0.386096,0.051130,0.382454,0.060840,0.421280,0.109433,0.471391,0.079143,...,0.110485,0.529955,0.122939,0.261760,0.175861,2.214848,1578.117554,-0.300794,0.122395,0.162162
1,0.457511,0.036216,0.386537,0.054978,0.383161,0.065258,0.418613,0.114704,0.472604,0.084320,...,0.123364,0.534689,0.134590,0.276021,0.175503,0.717034,16.020710,-0.310566,0.121469,0.774194
2,0.455678,0.033370,0.384117,0.050968,0.380257,0.060536,0.418308,0.109502,0.469279,0.078970,...,0.110045,0.525931,0.123548,0.266516,0.177869,2.230171,1578.903687,-0.304839,0.123555,0.421053
3,0.455079,0.039118,0.383206,0.055043,0.378348,0.063841,0.407803,0.114215,0.464298,0.083175,...,0.110052,0.516630,0.126275,0.294923,0.182257,1.333029,568.569153,-0.323153,0.124331,0.459459
4,0.457388,0.051691,0.383869,0.065948,0.379164,0.073005,0.405273,0.122115,0.468420,0.092958,...,0.126575,0.524533,0.136418,0.318795,0.177538,0.857873,15.257426,-0.342448,0.118246,0.344828
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8645,0.464178,0.025585,0.425263,0.038054,0.457938,0.052410,0.538639,0.087195,0.605670,0.079078,...,0.145029,0.747932,0.129702,0.212414,0.061206,0.400252,0.148672,-0.285587,0.043314,1.000000
8646,0.492750,0.035568,0.453915,0.051316,0.487138,0.065434,0.576250,0.101419,0.633773,0.090945,...,0.142710,0.798698,0.140896,0.189413,0.064255,0.368493,0.344626,-0.266063,0.048680,0.803279
8647,0.477754,0.028659,0.439266,0.042295,0.477514,0.057077,0.578447,0.092484,0.640532,0.085259,...,0.127509,0.811028,0.127238,0.198552,0.055144,0.360158,0.125844,-0.286069,0.041514,1.000000
8648,0.450301,0.030846,0.403263,0.043104,0.429319,0.057291,0.497814,0.097993,0.566541,0.086981,...,0.140758,0.706135,0.136380,0.250761,0.079156,0.485403,0.207889,-0.314607,0.048286,1.000000


In [12]:
train_df['deprived_sev']

0        0.162162
1        0.774194
2        0.421053
3        0.459459
4        0.344828
           ...   
24209    1.000000
24210    0.803279
24211    1.000000
24212    1.000000
24213    1.000000
Name: deprived_sev, Length: 8650, dtype: float64

In [13]:
df['y']

0       0.162162
1       0.774194
2       0.421053
3       0.459459
4       0.344828
          ...   
8645    1.000000
8646    0.803279
8647    1.000000
8648    1.000000
8649    1.000000
Name: y, Length: 8650, dtype: float64

In [7]:
# Import necessary libraries
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, r2_score, make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, cross_validate, KFold
import numpy as np

# Define a custom scorer for MAE since cross_val_score by default expects higher scores to be better
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

# Read the data from .npy files
X = np.load('X.npy')
y = np.load('y.npy')

# Select relevant feature columns (columns 2 to 7)
# X = X[:, 2:8]

# Initialize the scaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Initialize the regressor (Random Forest in this case)
regressor = RandomForestRegressor(random_state=42)

# Define the cross-validation strategy (e.g., 5-fold cross-validation)
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation
# Using cross_validate to obtain both MAE and R^2 scores
cv_results = cross_validate(
    regressor, 
    X_scaled, 
    y, 
    cv=cv, 
    scoring={'MAE': mae_scorer, 'R2': 'r2'},
    return_train_score=False
)

# Extract MAE and R2 scores
# Note: MAE scores are negative because greater_is_better=False in the scorer
mae_scores = -cv_results['test_MAE']
r2_scores = cv_results['test_R2']

# Calculate mean and standard error for MAE
mae_mean = mae_scores.mean()
mae_std_error = mae_scores.std(ddof=1) / np.sqrt(len(mae_scores))

# Calculate mean and standard error for R2
r2_mean = r2_scores.mean()
r2_std_error = r2_scores.std(ddof=1) / np.sqrt(len(r2_scores))

# Print the cross-validated results
print(f"Cross-Validated MAE: {mae_mean:.4f} ± {mae_std_error:.4f}")
print(f"Cross-Validated R²: {r2_mean:.4f} ± {r2_std_error:.4f}")

Cross-Validated MAE: 0.1924 ± 0.0012
Cross-Validated R²: 0.4675 ± 0.0088


In [18]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np


# Read the data from .npy files
X = np.load('X.npy')
y = np.load('y.npy')
X = X[:, 2:8]
#concant X and y
df = pd.DataFrame(X)
df['y'] = y
df['year'] = train_df['YEAR']

X_train = df.drop(columns=['y', 'year'])[df['year'] < 2020]
y_train = df['y'][df['year'] < 2020]
X_test = df.drop(columns=['y', 'year'])[df['year'] >= 2020]
y_test = df['y'][df['year'] >= 2020]

# Define the machine learning pipeline
pipeline = make_pipeline(
    StandardScaler(),
    RandomForestRegressor(random_state=42)
)

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

# Calculate evaluation metrics
test_mae = mean_absolute_error(y_test, y_pred)
test_r2 = r2_score(y_test, y_pred)

# Print test set results
print(f"Test MAE: {test_mae:.4f}")
print(f"Test R²: {test_r2:.4f}")

Test MAE: 0.2671
Test R²: 0.0418
